In [1]:
import cv2
import os
import numpy as np

def segmentsbce():
    print("Segmenting")
    # set paths
    input_path = os.path.join(os.getcwd(), 'good/')
    output_path = os.path.join(os.getcwd(), 'extracttest/')
    
    # loop over all image files in the 'good' folder
    for filename in os.listdir(input_path):
        # read input image
        inputfile = cv2.imread(os.path.join(input_path, filename))

        # shadow removal
        rgb_planes=cv2.split(inputfile)
        result_planes=[]
        result_norm_planes=[]
        kernel = np.ones((7,7),np.uint8)
        for plane in rgb_planes:
            dilated_img=cv2.dilate(plane,kernel)
            bg_img=cv2.medianBlur(dilated_img,21) # scratch text out
            diff_img=255-cv2.absdiff(plane,bg_img)
            norm_img=cv2.normalize(diff_img,diff_img,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8UC1)
            result_planes.append(diff_img)
            result_norm_planes.append(norm_img)
        result=cv2.merge(result_norm_planes)

        # line segmentation
        gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0)
        ret, binary = cv2.threshold(blur,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        binary = 255 - binary

        # perform connected component analysis
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8, ltype=cv2.CV_32S)

        for i in range(1, num_labels):
            # get the bounding box of the component
            x, y, w, h, _ = stats[i]
            char = binary[y:y+h,x:x+w]

            # save characters
            cv2.imwrite(os.path.join(output_path, '{}_char_{}.jpg'.format(os.path.splitext(filename)[0], i)), char)

    print("Segmentation complete.")

segmentsbce()

Segmenting
Segmentation complete.


In [ ]:
# binarizing - untitled 2